In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import csv
import re

In [ ]:
!wget -O singlerp.csv https://www.dropbox.com/s/y6zmuirq4mwdlgn/singlerp.csv?dl=0

In [ ]:
!wget -O rad.txt https://www.dropbox.com/s/npg42xdq2lzemnn/KS185_20200918_114153_Rad.txt?dl=0

In [ ]:
id = pd.read_csv('singlerp.csv', header=0, low_memory=False) # Why only single but not multiple?

In [ ]:
id.head()

,EMPI,date_rp,hospital_rp,lap_rp,lymph_rp
0,100126964,2019-03-25,MGH,1,0
1,100103172,2019-10-16,MGH,1,0
2,100244916,2018-11-02,MGH,1,1
3,100229964,2017-07-24,MGH,1,1
4,100081580,2016-09-23,MGH,1,1


In [ ]:
!wget -O pat1.txt https://www.dropbox.com/s/k26bb5ewsx71n88/KS185_20200918_114153_Pat.txt?dl=0

In [ ]:
!wget -O pat2.csv https://www.dropbox.com/s/cihhaunbfk1nzp1/Pat.csv?dl=0

In [ ]:
!wget -O pat3.csv https://www.dropbox.com/s/t2th5b5iuvz72zk/Pat.csv?dl=0

In [ ]:
!wget -O pat4.csv https://www.dropbox.com/s/v24o2fkl3t07ahx/Pat.csv?dl=0

In [ ]:
df2 = pd.read_csv('pat2.csv',sep="|", header=0, low_memory=False)

In [ ]:
df3 = pd.read_csv('pat3.csv',sep="|", header=0, low_memory=False)

In [ ]:
df4 = pd.read_csv('pat4.csv',sep="|", header=0, low_memory=False)

In [ ]:
# Check 1st line of file
rows = []
curr_row = []
with open('pat1.txt', encoding='utf-8') as input_file:
    
    first_line = input_file.readline()
    
    # Skip processing if current file doens't have multiline reports, just copy original txt file as csv
    if "|Report" not in first_line:
        shutil.copyfile('pat1.txt','pat1.csv')
        input_file.close()

    else:
        input_file.close()

for line in open('pat1.txt', encoding='utf-8'):

    # Header of the file
    if "EPIC_PMRN|" in line:
        rows.append(line)
        curr_row = []

    # Beginning line for each record
    elif '|' in line:
        curr_row.append(line)

    # End line for each record
    elif '[report_end]' in line:
        rows.append(curr_row)
        curr_row = []

    # data row for each record, likely report text
    else:
        curr_row.append(line)

# Write to output file
output_file = open('pat1.csv','w',newline='',encoding='utf-8')

def strip_whitespace(record):
    return " ".join(record.split())
      
for r in rows:

    curr_record = ''.join(r)
    curr_record = strip_whitespace(curr_record)
    output_file.write(curr_record + '\n')

output_file.close()

In [ ]:
df1 = pd.read_csv('pat1.csv',sep="|", low_memory=False)

In [ ]:
df1.head()

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Date_Time,Report_Description,Report_Status,Report_Type,Report_Text
0,100005276,1.002419e+10,FH,366628,S04-5802,7/8/2004 12:00:00 AM,Surgical Pathology,Signed,PAT,Accession Number: S04-5802 Report Status: Sign...
1,100005276,1.002419e+10,FH,366628,S04-900,1/29/2004 12:00:00 AM,Surgical Pathology,Signed,PAT,Accession Number: S04-900 Report Status: Signe...
2,100005276,1.002419e+10,FH,366628,S03-6789,9/2/2003 12:00:00 AM,Surgical Pathology,Signed,PAT,Accession Number: S03-6789 Report Status: Sign...
3,100005636,1.004004e+10,MGH,432959,S96E33362,8/27/1996 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: S96E33362 Report Status: Fin...
4,100005636,1.004004e+10,BWH,882951,S8810652D,8/22/1988 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: S8810652D Report Status: Fin...


In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(104984, 10)
(54948, 10)
(104141, 10)
(103843, 10)


In [ ]:
df1 = df1[df1['EMPI'].isin(id.EMPI)]
df2 = df2[df2['EMPI'].isin(id.EMPI)]
df3 = df3[df3['EMPI'].isin(id.EMPI)]
df4 = df4[df4['EMPI'].isin(id.EMPI)]

In [ ]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(24878, 10)
(12663, 10)
(25033, 10)
(24605, 10)


In [ ]:
df1

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Date_Time,Report_Description,Report_Status,Report_Type,Report_Text
5,100003956,1.002650e+10,MGH,1784307,S02K17074,3/29/2002 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: S02K17074 Report Status: Fin...
6,100003956,1.002650e+10,MGH,1784307,S02M6053,1/31/2002 2:05:00 PM,Surgical Pathology,Final,PAT,Accession Number: S02M6053 Report Status: Fina...
7,100003956,1.002650e+10,NWH,265017,S01:014639,11/13/2001 12:00:00 AM,Surgical Pathology,Signed,PAT,Accession Number: S01:014639 Report Status: Si...
8,100003956,1.002650e+10,NWH,265017,C01:020125,10/30/2001 12:00:00 AM,Cytology,Signed,PAT,Accession Number: C01:020125 Report Status: Si...
9,100003956,1.002650e+10,NWH,265017,U99:000131,1/11/1999 2:41:00 PM,Surgical Pathology,Signed,PAT,Accession Number: U99:000131 Report Status: Si...
...,...,...,...,...,...,...,...,...,...,...
104979,111036658,1.010503e+10,BWH,30970941,BS16G47035,9/1/2016 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS16G47035 Report Status: Fi...
104980,111036658,1.010503e+10,BWH,30970941,BS15X53915,11/5/2015 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS15X53915 Report Status: Fi...
104981,111036658,1.010503e+10,BWH,30970941,BS15N38425,7/10/2015 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS15N38425 Report Status: Fi...
104982,111049658,1.010516e+10,BWH,31044431,BS15F39420,7/24/2015 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS15F39420 Report Status: Fi...


In [ ]:
id[id.EMPI == 	100003956] 

,EMPI,date_rp,hospital_rp,lap_rp,lymph_rp
14,100003956,2002-03-29,MGH,0,0


In [ ]:
df1[df1.EMPI == 100003956]

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Date_Time,Report_Description,Report_Status,Report_Type,Report_Text
5,100003956,1.002650e+10,MGH,1784307,S02K17074,3/29/2002 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: S02K17074 Report Status: Fin...
6,100003956,1.002650e+10,MGH,1784307,S02M6053,1/31/2002 2:05:00 PM,Surgical Pathology,Final,PAT,Accession Number: S02M6053 Report Status: Fina...
7,100003956,1.002650e+10,NWH,265017,S01:014639,11/13/2001 12:00:00 AM,Surgical Pathology,Signed,PAT,Accession Number: S01:014639 Report Status: Si...
8,100003956,1.002650e+10,NWH,265017,C01:020125,10/30/2001 12:00:00 AM,Cytology,Signed,PAT,Accession Number: C01:020125 Report Status: Si...
9,100003956,1.002650e+10,NWH,265017,U99:000131,1/11/1999 2:41:00 PM,Surgical Pathology,Signed,PAT,Accession Number: U99:000131 Report Status: Si...


In [ ]:
df1[df1.EMPI == 100003956].Report_Text.iloc[0] 

'Accession Number: S02K17074 Report Status: Final Type: Surgical Pathology Pathology Report: S02-K17074 OPERATION DATE: 29 MAR 02 ACCESSIONED ON: 29 MAR 02 at 12:56 CLINICAL DATA: Prostate cancer. =============================================================================== *** FINAL DIAGNOSIS *** PROSTATE, EXCISION: ADENOCARCINOMA, GLEASON SCORE 3+4=7/10, CONFINED TO THE PROSTATE. See note. NOTE: The tumor involves predominantly left and right posterior quadrants and focally left anterior quadrant. All resection margins are negative for tumor. Tumor does not extend into periprostatic fat. Perineural invasion is present. No lymphatic or vascular invasion is seen. The seminal vesicles and vasa deferentia are unremarkable. LYMPH NODES (LEFT AND RIGHT), EXCISION: THERE IS NO EVIDENCE OF MALIGNANCY IN 2 LYMPH NODES. THE FROZEN SECTION DIAGNOSIS IS CONFIRMED. Diagnosis by: Robert B. Colvin, M.D. Signed On: 02 APR 02 at 18:18 ================================================================

In [ ]:
df1[df1.EMPI == 100003956].Report_Text.iloc[1]

'Accession Number: S02M6053 Report Status: Final Type: Surgical Pathology Pathology Report: S02-M6053 ACCESSIONED ON: 31 JAN 02 at 14:05 CLINICAL DATA: Elevated PSA; BPH. =============================================================================== *** FINAL DIAGNOSIS *** PROSTATE, RIGHT APEX (S01:014639-A): ADENOCARCINOMA, GLEASON SCORE 3+3= 6/10, INVOLVING 1 OF 2 TISSUE CORES AND 10% OF TOTAL TISSUE VOLUME. RIGHT, RIGHT MID (SO1:0146339-B): ADENOCARCINOMA, GLEASON GRADE 3+3=SCORE 6/10, INVOLVING 1 OF 2 TISSUE CORES AND <5% OF TOTAL TISSUE VOLUME. PROSTATE, RIGHT BASE (S01:014639-C): BENIGN PROSTATIC TISSUE. PROSTATE, LEFT APEX (S01:014639-D): ADENOCARCINOMA, GLEASON GRADE 3/5, ONE MICROSCOPIC FOCUS INVOLVING 1 OF 4 TISSUE CORE FRAGMENTS. PROSTATE, LEFT MID (S01:014639-E): ADENOCARCINOMA, GLEASON GRADE 3/5, ONE MICROSCOPIC FOCUS INVOLVING 1 OF 2 TISSUE CORES. PROSTATE, LEFT BASE (S01:01469-F): ADENOCARCINOMA, GLEASON SCORE 4+3=7/10, INVOLVING 1 OF 3 TISSUE CORE FRAGMENTS AND 5% OF T

In [ ]:
id[id.EMPI == 	111036658]

,EMPI,date_rp,hospital_rp,lap_rp,lymph_rp
4810,111036658,2015-11-05,BWH,1,1


In [ ]:
df1[df1.EMPI == 111036658]

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Date_Time,Report_Description,Report_Status,Report_Type,Report_Text
104979,111036658,1.010503e+10,BWH,30970941,BS16G47035,9/1/2016 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS16G47035 Report Status: Fi...
104980,111036658,1.010503e+10,BWH,30970941,BS15X53915,11/5/2015 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS15X53915 Report Status: Fi...
104981,111036658,1.010503e+10,BWH,30970941,BS15N38425,7/10/2015 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS15N38425 Report Status: Fi...


In [ ]:
df1.dtypes

EMPI                    int64
EPIC_PMRN             float64
MRN_Type               object
MRN                     int64
Report_Number          object
Report_Date_Time       object
Report_Description     object
Report_Status          object
Report_Type            object
Report_Text            object
dtype: object

In [ ]:
df_merged = df1.append([df2, df3, df4])

In [ ]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87179 entries, 5 to 103842
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EMPI                87179 non-null  int64  
 1   EPIC_PMRN           86492 non-null  float64
 2   MRN_Type            87179 non-null  object 
 3   MRN                 87179 non-null  int64  
 4   Report_Number       87179 non-null  object 
 5   Report_Date_Time    87179 non-null  object 
 6   Report_Description  87179 non-null  object 
 7   Report_Status       87179 non-null  object 
 8   Report_Type         87179 non-null  object 
 9   Report_Text         87179 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.3+ MB


In [ ]:
df_merged["Report_Text"] = df_merged["Report_Text"].str.lower()

In [ ]:
df_merged['date_pathology'] = pd.to_datetime(df_merged['Report_Date_Time'], infer_datetime_format=True)

In [ ]:
df_merged.head()

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Date_Time,Report_Description,Report_Status,Report_Type,Report_Text,date_pathology
5,100003956,1.002650e+10,MGH,1784307,S02K17074,3/29/2002 12:00:00 AM,Surgical Pathology,Final,PAT,accession number: s02k17074 report status: fin...,2002-03-29 00:00:00
6,100003956,1.002650e+10,MGH,1784307,S02M6053,1/31/2002 2:05:00 PM,Surgical Pathology,Final,PAT,accession number: s02m6053 report status: fina...,2002-01-31 14:05:00
7,100003956,1.002650e+10,NWH,265017,S01:014639,11/13/2001 12:00:00 AM,Surgical Pathology,Signed,PAT,accession number: s01:014639 report status: si...,2001-11-13 00:00:00
8,100003956,1.002650e+10,NWH,265017,C01:020125,10/30/2001 12:00:00 AM,Cytology,Signed,PAT,accession number: c01:020125 report status: si...,2001-10-30 00:00:00
9,100003956,1.002650e+10,NWH,265017,U99:000131,1/11/1999 2:41:00 PM,Surgical Pathology,Signed,PAT,accession number: u99:000131 report status: si...,1999-01-11 14:41:00


In [ ]:
df_merged.drop(['Report_Date_Time'], axis=1, inplace=True)

In [ ]:
df_merged['Report_Description'].value_counts()

Surgical Pathology       74056
Cytology                 10802
Interpretive Lab Test     1750
Cytogenetics               360
Clinical Pathology          91
Autopsy                     81
Pathology Report            39
Name: Report_Description, dtype: int64

In [ ]:
surgical_path = df_merged.loc[df_merged.Report_Description == 'Surgical Pathology']

In [ ]:
surgical_path.head()

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Description,Report_Status,Report_Type,Report_Text,date_pathology
5,100003956,1.002650e+10,MGH,1784307,S02K17074,Surgical Pathology,Final,PAT,accession number: s02k17074 report status: fin...,2002-03-29 00:00:00
6,100003956,1.002650e+10,MGH,1784307,S02M6053,Surgical Pathology,Final,PAT,accession number: s02m6053 report status: fina...,2002-01-31 14:05:00
7,100003956,1.002650e+10,NWH,265017,S01:014639,Surgical Pathology,Signed,PAT,accession number: s01:014639 report status: si...,2001-11-13 00:00:00
9,100003956,1.002650e+10,NWH,265017,U99:000131,Surgical Pathology,Signed,PAT,accession number: u99:000131 report status: si...,1999-01-11 14:41:00
32,100006532,1.004006e+10,BWH,975151,BS19D21021,Surgical Pathology,Final,PAT,accession number: bs19d21021 report status: fi...,2019-04-10 00:00:00


In [ ]:
surgical_path['date'] = pd.to_datetime(surgical_path['date_pathology']).dt.date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
surgical_path.date = pd.to_datetime(surgical_path.date)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
id['date_pathology'] = pd.to_datetime(id['date_rp'], infer_datetime_format=True)

In [ ]:
final_df = surgical_path.merge(id,left_on=['EMPI','date'], right_on = ['EMPI','date_pathology'])

In [ ]:
final_df

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Description,Report_Status,Report_Type,Report_Text,date_pathology_x,date,date_rp,hospital_rp,lap_rp,lymph_rp,date_pathology_y
0,100003956,1.002650e+10,MGH,1784307,S02K17074,Surgical Pathology,Final,PAT,accession number: s02k17074 report status: fin...,2002-03-29 00:00:00,2002-03-29,2002-03-29,MGH,0,0,2002-03-29
1,100006532,1.004006e+10,BWH,975151,S9905462C,Surgical Pathology,Final,PAT,accession number: s9905462c report status: fin...,1999-02-27 00:00:00,1999-02-27,1999-02-27,BWH,0,0,1999-02-27
2,100006636,1.004006e+10,BWH,984781,BS05E05528,Surgical Pathology,Final,PAT,accession number: bs05e05528 report status: fi...,2005-02-10 00:00:00,2005-02-10,2005-02-10,BWH,0,1,2005-02-10
3,100001308,1.004001e+10,BWH,279307,S9935420S,Surgical Pathology,Final,PAT,accession number: s9935420s report status: fin...,1999-12-17 00:00:00,1999-12-17,1999-12-17,BWH,0,0,1999-12-17
4,100095964,1.004423e+10,MGH,3408237,S04T31383,Surgical Pathology,Final,PAT,accession number: s04t31383 report status: fin...,2004-06-02 00:00:00,2004-06-02,2004-06-02,MGH,0,0,2004-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16651,111108962,1.010778e+10,MGH,5708043,S17-42561,Surgical Pathology,Final,PAT,accession number: s17-42561 report status: fin...,2017-06-23 00:00:00,2017-06-23,2017-06-23,MGH,1,1,2017-06-23
16652,111110994,1.010780e+10,MGH,5703546,S15-80722,Surgical Pathology,Final,PAT,accession number: s15-80722 report status: fin...,2015-12-15 00:00:00,2015-12-15,2015-12-15,MGH,1,0,2015-12-15
16653,111085770,1.010554e+10,MGH,5690246,S19-42440,Surgical Pathology,Final,PAT,accession number: s19-42440 report status: fin...,2019-06-18 14:48:00,2019-06-18,2019-06-18,MGH,1,1,2019-06-18
16654,111142034,1.011387e+10,MGH,1326008,S19-56696,Surgical Pathology,Final,PAT,accession number: s19-56696 report status: fin...,2019-08-14 00:00:00,2019-08-14,2019-08-14,MGH,0,0,2019-08-14


In [ ]:
final_df = final_df[['EMPI','Report_Text','hospital_rp','date']]

In [ ]:
final_df.to_csv('X_data.csv',index=False)

In [ ]:
final_df.EMPI.value_counts()

114192005    4
114311515    3
107440854    3
103938133    3
113873044    3
            ..
104299197    1
104292797    1
104265245    1
104246725    1
111061498    1
Name: EMPI, Length: 16494, dtype: int64

In [ ]:
id.shape

(16768, 6)

In [ ]:
final_df.columns # hospital/id/date/text/label

Index(['EMPI', 'Report_Text', 'hospital_rp', 'date'], dtype='object')

In [ ]:
final_df[final_df.EMPI == 114192005].Report_Text.iloc[0]

'accession number: s19-6963 report status: updated type: surgical pathology procedure date: 01/30/2019 ordering provider: kim, michelle massachusetts general hospital 55 fruit street, boston, ma 02114 tel (617) 726-2967 surgical pathology report accession #: s19-6963 patient name: maddocks, joseph dob: 1/12/1966 (age: 53) sex: m mrn: 6547548 location: periop institution: mgh date of operation: 1/30/2019 date of accession: 1/30/2019 reported: 2/8/2019 18:58 results to: michelle m kim md, phd final pathologic diagnosis: a. ureter biopsy, left distal: - there is no evidence of malignancy. b. ureter biopsy, right distal: - there is no evidence of malignancy. c. bladder and prostate cystoprostatectomy: - urothelial carcinoma, high-grade, with extension into perivesicular adipose, seminal vesicle, prostate, and bowel wall (part g), see synoptic report #1. - lymphovascular invasion is present. - prostatic adenocarcinoma, gleason score 3 + 3 = 6/10 (grade group 1), confined to the gland, see s

In [ ]:
final_df[final_df.EMPI == 114192005].Report_Text.iloc[1]

'accession number: m19-1042 report status: final type: surgical pathology procedure date: 01/30/2019 ordering provider: kim, michelle massachusetts general hospital boston, ma 02114 tel (617) 726-2967 molecular pathology report solid snapshot assay accession #: m19-1042 patient name: maddocks, joseph dob: 1/12/1966 (age: 53) sex: m mrn: 6547548 institution: mgh location: mgh e06 date of collection: 1/30/2019 date of accession: 2/1/2019 reported: 2/19/2019 17:35 results to: michelle m kim md, phd xin gao md clinical history: bladder high grade urothelial carcinoma specimens received: a: molecular test for solid snapshot assay gross description: 1 block from massachusetts general hospital; the archived specimen was retrieved and reviewed for adequacy slide-block description: s19-6963 c11 test - snapshot-ngs-v2 assay indication for test: bladder high grade urothelial carcinoma specimen(s) tested: s19-6963 c11 (massachusetts general hospital, boston, ma, united states) results: single nucl

In [ ]:
final_df[final_df.EMPI == 114192005].Report_Text.iloc[2]

'accession number: m19-1043 report status: final type: surgical pathology procedure date: 01/30/2019 ordering provider: kim, michelle massachusetts general hospital boston, ma 02114 tel (617) 726-2967 molecular pathology report solid fusion assay accession #: m19-1043 patient name: maddocks, joseph dob: 1/12/1966 (age: 53) sex: m mrn: 6547548 institution: mgh location: mgh e06 date of collection: 1/30/2019 date of accession: 2/1/2019 reported: 2/22/2019 20:27 results to: michelle m kim md, phd xin gao md clinical history: bladder high grade urothelial carcinoma specimens received: a: molecular test for solid fusion assay gross description: 1 block from massachusetts general hospital; the archived specimen was retrieved and reviewed for adequacy slide-block description: s19-6963 c11 test - solid fusion assay v2 indication for test: bladder high grade urothelial carcinoma specimen(s) tested: s19-6963 c11 (massachusetts general hospital, boston, ma, united states) results: targeted rna ne

In [ ]:
final_df

,EMPI,Report_Text,hospital_rp,date
0,100003956,accession number: s02k17074 report status: fin...,MGH,2002-03-29
1,100006532,accession number: s9905462c report status: fin...,BWH,1999-02-27
2,100006636,accession number: bs05e05528 report status: fi...,BWH,2005-02-10
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17
4,100095964,accession number: s04t31383 report status: fin...,MGH,2004-06-02
...,...,...,...,...
16651,111108962,accession number: s17-42561 report status: fin...,MGH,2017-06-23
16652,111110994,accession number: s15-80722 report status: fin...,MGH,2015-12-15
16653,111085770,accession number: s19-42440 report status: fin...,MGH,2019-06-18
16654,111142034,accession number: s19-56696 report status: fin...,MGH,2019-08-14


Below is the operation to selecte people with metasis

In [ ]:
keep_terms_meta = ['n1','carcinoma involving']
keep_terms = ['t2b','plnd','SPECIMEN TYPE: Lymph node biopsy']
remove_terms_meta = ['no carcinoma']

In [ ]:
for i in range(len(keep_terms)):
    keep_terms[i] = keep_terms[i].lower()

In [ ]:
final_df[final_df['Report_Text'].str.contains('|'.join(keep_terms + keep_terms_meta), case=False)]

,EMPI,Report_Text,hospital_rp,date
1,100006532,accession number: s9905462c report status: fin...,BWH,1999-02-27
2,100006636,accession number: bs05e05528 report status: fi...,BWH,2005-02-10
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17
11,100086908,accession number: bs08a16092 report status: fi...,BWH,2008-04-14
12,100126964,accession number: s19-20098 report status: fin...,MGH,2019-03-25
...,...,...,...,...
16620,109123082,accession number: s14-38753 report status: fin...,MGH,2014-06-17
16623,109728042,accession number: bs08w26932 report status: fi...,BWH,2008-06-19
16629,109969370,accession number: bs15f29649 report status: fi...,BWH,2015-06-18
16636,109979050,accession number: bs14a59048 report status: fi...,BWH,2014-11-18


In [ ]:
final_df[final_df['Report_Text'].str.contains('|'.join(keep_terms_meta), case=False)]

,EMPI,Report_Text,hospital_rp,date
12,100126964,accession number: s19-20098 report status: fin...,MGH,2019-03-25
80,100003779,accession number: s95n17653 report status: fin...,MGH,1995-05-22
86,100076587,accession number: s95d8539 report status: fina...,MGH,1995-03-13
92,100138531,accession number: bs19f04711 report status: up...,BWH,2019-01-24
111,100262051,accession number: ms06n16738 report status: fi...,MGH,2006-03-29
...,...,...,...,...
16513,102028850,accession number: s0226715r report status: fin...,BWH,2002-08-07
16542,102485346,accession number: s19-16527 report status: fin...,MGH,2019-03-11
16577,113351226,accession number: bs19a52508 report status: fi...,BWH,2019-09-06
16589,114288418,accession number: fs19k05224 report status: fi...,FH,2019-05-24


In [ ]:
final_df[final_df['Report_Text'].str.contains('|'.join(remove_terms_meta), case=False)]

,EMPI,Report_Text,hospital_rp,date
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17
13,100111708,accession number: s0013104l report status: fin...,BWH,2000-05-05
24,100229964,accession number: s17-49263 report status: fin...,MGH,2017-07-24
28,100239292,accession number: s9922464c report status: fin...,BWH,1999-08-12
35,100200108,accession number: s9824819s report status: fin...,BWH,1998-09-22
...,...,...,...,...
16557,102522906,accession number: s0223052h report status: fin...,BWH,2002-07-09
16559,102533602,accession number: s0238744n report status: fin...,BWH,2002-11-14
16566,102753898,accession number: s0325336t report status: fin...,BWH,2003-07-22
16571,102631330,accession number: s0241853l report status: fin...,BWH,2002-12-10


In [ ]:
final_df[~((final_df['Report_Text'].str.contains('lymph')) & (final_df['Report_Text'].str.contains('confined')))].Report_Text.iloc[0]

'accession number: s9905462c report status: final type: surgical pathology pathology report s99-05462-c lymph nodes regional resection accessioned on: 02/27/99 longtine, janina, a., m.d. diagnosis: by longtine,janina a.,m.d. 1) specimen labeled "left obturator node" (including fsa1, fsa2, fsa3): twelve (12) lymph nodes with no tumor seen. 2) specimen labeled "right obturator node" (including fsb1, fsb2): ten (10) lymph nodes with no tumor seen. 3) specimen labeled "prostate, radical prostatectomy": prostatic adenocarcinoma, gleason grade 3+4, involving both lobes, 0.9 cm. in greatest single dimension. the margins of resection are free of tumor. perineural invasion is present. tumor is not identified in extraprostatic soft tissue. the seminal vesicles are free of tumor. ajcc (1997) stage t2b n0 mx. clinical data: history: a 56-year-old male with prostatic adenocarcinoma, gleason 3+4, in one of three (1:3) cores (20% tissue), on left side. tissue submitted: 1) left obturator node 2) righ

In [ ]:
final_df[final_df['Report_Text'].str.contains('prostatectomy')]

,EMPI,Report_Text,hospital_rp,date
0,100003956,accession number: s02k17074 report status: fin...,MGH,2002-03-29
1,100006532,accession number: s9905462c report status: fin...,BWH,1999-02-27
2,100006636,accession number: bs05e05528 report status: fi...,BWH,2005-02-10
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17
4,100095964,accession number: s04t31383 report status: fin...,MGH,2004-06-02
...,...,...,...,...
16651,111108962,accession number: s17-42561 report status: fin...,MGH,2017-06-23
16652,111110994,accession number: s15-80722 report status: fin...,MGH,2015-12-15
16653,111085770,accession number: s19-42440 report status: fin...,MGH,2019-06-18
16654,111142034,accession number: s19-56696 report status: fin...,MGH,2019-08-14


In [ ]:
final_df.shape

(16656, 4)

In [ ]:
# PLND labeling
searchforPLND = ["lymph"]
lymph_df = final_df[final_df['Report_Text'].str.contains('|'.join(searchforPLND),case=False)]
display(lymph_df.shape)
display(lymph_df.Report_Text.iloc[0])
display(lymph_df[~lymph_df['Report_Text'].str.contains("node",case=False)].Report_Text.iloc[1])

# remove duplicate patients based on non-unique EMPI
lymph_df = lymph_df.drop_duplicates(subset=['EMPI'], keep=False)

lymph_df

(14597, 4)

'accession number: s02k17074 report status: final type: surgical pathology pathology report: s02-k17074 operation date: 29 mar 02 accessioned on: 29 mar 02 at 12:56 clinical data: prostate cancer. =============================================================================== *** final diagnosis *** prostate, excision: adenocarcinoma, gleason score 3+4=7/10, confined to the prostate. see note. note: the tumor involves predominantly left and right posterior quadrants and focally left anterior quadrant. all resection margins are negative for tumor. tumor does not extend into periprostatic fat. perineural invasion is present. no lymphatic or vascular invasion is seen. the seminal vesicles and vasa deferentia are unremarkable. lymph nodes (left and right), excision: there is no evidence of malignancy in 2 lymph nodes. the frozen section diagnosis is confirmed. diagnosis by: robert b. colvin, m.d. signed on: 02 apr 02 at 18:18 ================================================================

'accession number: bs08n45729 report status: final type: surgical pathology specimen type: prostate, radical resection procedure date: 10/16/2008 ordering provider: jim c hu m.d., m.p.h. case: bs-08-n45729 patient: scott loos resident: christian j. davidson, m.d. pathologist: lucian r chirieac, m.d. pathologic diagnosis: specimen designated "prostate": prostatic adenocarcinoma (2.4 cm in greatest dimension), gleason score 3+3 =6, present in both sides. no lymphovascular invasion is identified. tumor is not present in extraprostatic soft tissue. tumor is not present in the seminal vesicles. tumor is not present at the soft tissue resection margin. multiple levels examined. ajcc stage (6th edition): t2c nx mx. clinical data: history: none specified. operation: robotic prostatectomy. operative findings: none specified. clinical diagnosis: prostate ca. tissue submitted: a/1) prostate (s.s). gross description: specimen a, labeled "#1. prostate", is received fresh in a specimen container, cl

,EMPI,Report_Text,hospital_rp,date
0,100003956,accession number: s02k17074 report status: fin...,MGH,2002-03-29
1,100006532,accession number: s9905462c report status: fin...,BWH,1999-02-27
2,100006636,accession number: bs05e05528 report status: fi...,BWH,2005-02-10
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17
5,100093404,accession number: s99t17262 report status: fin...,MGH,1999-04-14
...,...,...,...,...
16651,111108962,accession number: s17-42561 report status: fin...,MGH,2017-06-23
16652,111110994,accession number: s15-80722 report status: fin...,MGH,2015-12-15
16653,111085770,accession number: s19-42440 report status: fin...,MGH,2019-06-18
16654,111142034,accession number: s19-56696 report status: fin...,MGH,2019-08-14


In [ ]:
# PLND labeling - screening for false negatives (i.e. we label as PLND=0 when in fact they have PLND=1)
searchforPLND = ["lymph"]
noPLND_df = final_df[~final_df['Report_Text'].str.contains('|'.join(searchforPLND))]
display(noPLND_df[noPLND_df['Report_Text'].str.contains("malignan",case=False)].shape)
display(noPLND_df[noPLND_df['Report_Text'].str.contains("malignan",case=False)].Report_Text.iloc[1])

noPLNDyesNODE_df = noPLND_df[noPLND_df['Report_Text'].str.contains("node",case=False)]
display(noPLNDyesNODE_df[noPLNDyesNODE_df['Report_Text'].str.contains("obturator node",case=False)].shape)


(512, 4)

'accession number: s03m64703 report status: final type: surgical pathology pathology report: s03-m64703 operation date: 17 nov 03 accessioned on: 17 nov 03 at 16:42 clinical data: 59 year old male with prostate ca. =============================================================================== *** final diagnosis *** prostate gland, radical prostatectomy: prostatic adenocarcinoma, gleason score 3+3= 6/10, focally involving the right anterior quadrant and right posterior quadrant. extraprostatic extension is not present. perineural invasion is not seen. the seminal vesicles are unremarkable. all resection margins are negative. ajcc tnm pathologic stage (6th edition):pt2bnxmx. bladder (neck), excision: smooth muscle tissue. there is no evidence of malignancy. diagnosis by: chin-lee wu, md, phd signed on: 21 nov 03 at 17:49 =============================================================================== gross description: received fresh, labeled with the patient\'s name, unit number, and "

(11, 4)

In [ ]:
# Because AJCC staging (N0, NX, N1) could be embedded within accession number of reports, 
# remove accession number from pathology reports

lymph_df["Report_Text_noAN"] = lymph_df["Report_Text"].str.slice(28,None,1)
lymph_df.Report_Text_noAN

0        report status: final type: surgical pathology ...
1        report status: final type: surgical pathology ...
2         report status: final type: surgical pathology...
3        report status: final type: surgical pathology ...
5        report status: final type: surgical pathology ...
                               ...                        
16651    report status: final type: surgical pathology ...
16652    report status: final type: surgical pathology ...
16653    report status: final type: surgical pathology ...
16654    report status: final type: surgical pathology ...
16655     report status: final type: surgical pathology...
Name: Report_Text_noAN, Length: 14502, dtype: object

In [ ]:
lymph_df['LN_meta'] = pd.NaT
lymph_df['n0'] = pd.NaT
lymph_df['n1'] = pd.NaT
lymph_df['n2'] = pd.NaT
lymph_df['n3'] = pd.NaT

# identify patients with AJCC staging nx
searchfornx = ["nx ", "nxm", "nx\.", "nx,", "nx:", "nx\(", "nx\)"];
nx_df = lymph_df[lymph_df['Report_Text_noAN'].str.contains('|'.join(searchfornx))]
display("nx", nx_df.shape)

# remove patients who have PLND 0
lymph_df = pd.concat([lymph_df, nx_df]).drop_duplicates(keep=False)
display("lymph_df shape after removing patients with NX", lymph_df.shape)

# identify patients with AJCC staging n1, n2, n3
searchforn0 = ["n0 ", "n0m", "n0\.", "n0,", "n0:", "n0\(", "n0\)"]
n0_df = lymph_df[lymph_df['Report_Text_noAN'].str.contains('|'.join(searchforn0))]
display("n0", n0_df.shape)

searchforn1 = ["n1 ", "n1m", "n1\.", "n1,"]
n1_df = lymph_df[lymph_df['Report_Text_noAN'].str.contains('|'.join(searchforn1))]
display("n1", n1_df.shape)

searchforn2 = ["n2 ", "n2m", "n2\.", "n2,", "n2:", "n2\(", "n2\)"];
n2_df = lymph_df[lymph_df['Report_Text_noAN'].str.contains('|'.join(searchforn2))]
display("n2", n2_df.shape)

searchforn3 = ["n3 ", "n3m", "n3\.", "n3,", "n3:", "n3\(", "n3\)"];
n3_df = lymph_df[lymph_df['Report_Text_noAN'].str.contains('|'.join(searchforn3))]
display("n3", n3_df.shape)

# check overlap
n0n1 = n1_df[n1_df['Report_Text_noAN'].str.contains('|'.join(searchforn0))]
display("n0n1", n0n1.shape)
n0n2 = n2_df[n2_df['Report_Text_noAN'].str.contains('|'.join(searchforn0))]
display("n0n2", n0n2.shape)
n0n3 = n3_df[n3_df['Report_Text_noAN'].str.contains('|'.join(searchforn0))]
display("n0n3", n0n3.shape)
n1n2 = n2_df[n2_df['Report_Text_noAN'].str.contains('|'.join(searchforn1))]
display("n1n2", n1n2.shape)
n1n3 = n3_df[n3_df['Report_Text_noAN'].str.contains('|'.join(searchforn1))]
display("n1n3", n1n3.shape)
n2n3 = n3_df[n3_df['Report_Text_noAN'].str.contains('|'.join(searchforn2))]
display("n2n3", n2n3.shape)

# if there is multiple AJCC staging for a single patient, class that patient as 
# having no staging information
n0_df = pd.concat([n0_df, n0n1]).drop_duplicates(keep=False); #display(n0_df.shape)
n0_df = pd.concat([n0_df, n0n2]).drop_duplicates(keep=False); #display(n0_df.shape)
n0_df = pd.concat([n0_df, n0n3]).drop_duplicates(keep=False); #display(n0_df.shape)

n1_df = pd.concat([n1_df, n0n1]).drop_duplicates(keep=False); #display(n1_df.shape)
n1_df = pd.concat([n1_df, n1n2]).drop_duplicates(keep=False); #display(n1_df.shape)
n1_df = pd.concat([n1_df, n1n3]).drop_duplicates(keep=False); #display(n1_df.shape)

n2_df = pd.concat([n2_df, n0n2]).drop_duplicates(keep=False); #display(n2_df.shape)
n2_df = pd.concat([n2_df, n1n2]).drop_duplicates(keep=False); #display(n2_df.shape)
n2_df = pd.concat([n2_df, n2n3]).drop_duplicates(keep=False); #display(n2_df.shape)

n3_df = pd.concat([n3_df, n0n3]).drop_duplicates(keep=False); #display(n3_df.shape)
n3_df = pd.concat([n3_df, n1n3]).drop_duplicates(keep=False); #display(n3_df.shape)
n3_df = pd.concat([n3_df, n2n3]).drop_duplicates(keep=False); #display(n3_df.shape)

display("FINAL DF SHAPES", "n0", n0_df.shape, "n1", n1_df.shape,
        "n2", n2_df.shape, "n3", n3_df.shape)

# remove patients who have no staging
lymph_df_nostage = pd.concat([lymph_df, n0_df, n1_df, n2_df, n3_df]).drop_duplicates(keep=False)
# combine patients with staging
lymph_df_withstage = pd.concat([n0_df, n1_df, n2_df])
display(lymph_df_nostage.shape, lymph_df_withstage.shape)

lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n0_df.EMPI), "LN_meta"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n0_df.EMPI), "n0"] = 1
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n0_df.EMPI), "n1"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n0_df.EMPI), "n2"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n0_df.EMPI), "n3"] = 0

lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n1_df.EMPI), "LN_meta"] = 1
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n1_df.EMPI), "n0"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n1_df.EMPI), "n1"] = 1
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n1_df.EMPI), "n2"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n1_df.EMPI), "n3"] = 0

lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n2_df.EMPI), "LN_meta"] = 1
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n2_df.EMPI), "n0"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n2_df.EMPI), "n1"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n2_df.EMPI), "n2"] = 1
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n2_df.EMPI), "n3"] = 0

lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n3_df.EMPI), "LN_meta"] = 1
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n3_df.EMPI), "n0"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n3_df.EMPI), "n1"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n3_df.EMPI), "n2"] = 0
lymph_df_withstage.loc[lymph_df_withstage.EMPI.isin(n3_df.EMPI), "n3"] = 1

display(lymph_df)
display(lymph_df_nostage)
display(lymph_df_withstage)

'nx'

(0, 10)

'lymph_df shape after removing patients with NX'

(12167, 10)

'n0'

...

(7698, 10)

,EMPI,Report_Text,hospital_rp,date,Report_Text_noAN,LN_meta,n0,n1,n2,n3
0,100003956,accession number: s02k17074 report status: fin...,MGH,2002-03-29,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
1,100006532,accession number: s9905462c report status: fin...,BWH,1999-02-27,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
2,100006636,accession number: bs05e05528 report status: fi...,BWH,2005-02-10,report status: final type: surgical pathology...,NaT,NaT,NaT,NaT,NaT
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
5,100093404,accession number: s99t17262 report status: fin...,MGH,1999-04-14,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
16649,111104018,accession number: s15-66576 report status: fin...,MGH,2015-10-19,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
16650,111106474,accession number: ms06t48348 report status: fi...,MGH,2006-09-06,report status: final type: surgical pathology...,NaT,NaT,NaT,NaT,NaT
16651,111108962,accession number: s17-42561 report status: fin...,MGH,2017-06-23,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
16653,111085770,accession number: s19-42440 report status: fin...,MGH,2019-06-18,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT


,EMPI,Report_Text,hospital_rp,date,Report_Text_noAN,LN_meta,n0,n1,n2,n3
0,100003956,accession number: s02k17074 report status: fin...,MGH,2002-03-29,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
5,100093404,accession number: s99t17262 report status: fin...,MGH,1999-04-14,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
7,100104620,accession number: s95n33201 report status: fin...,MGH,1995-09-21,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
9,100074436,accession number: s03a71170 report status: fin...,MGH,2003-12-19,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
10,100076516,accession number: s94x36797 report status: fin...,MGH,1994-11-07,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
16605,111693306,accession number: s16-58580 report status: fin...,MGH,2016-09-06,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
16619,113078474,accession number: bs17w32233 report status: fi...,BWH,2017-06-07,report status: final type: surgical pathology...,NaT,NaT,NaT,NaT,NaT
16625,109714722,accession number: s03f36647 report status: fin...,MGH,2003-07-01,report status: final type: surgical pathology ...,NaT,NaT,NaT,NaT,NaT
16640,110530770,accession number: ms07d44309 report status: fi...,MGH,2007-07-23,report status: final type: surgical pathology...,NaT,NaT,NaT,NaT,NaT


,EMPI,Report_Text,hospital_rp,date,Report_Text_noAN,LN_meta,n0,n1,n2,n3
1,100006532,accession number: s9905462c report status: fin...,BWH,1999-02-27,report status: final type: surgical pathology ...,0,1,0,0,0
2,100006636,accession number: bs05e05528 report status: fi...,BWH,2005-02-10,report status: final type: surgical pathology...,0,1,0,0,0
3,100001308,accession number: s9935420s report status: fin...,BWH,1999-12-17,report status: final type: surgical pathology ...,0,1,0,0,0
8,100081580,accession number: s16-62959 report status: fin...,MGH,2016-09-23,report status: final type: surgical pathology ...,0,1,0,0,0
11,100086908,accession number: bs08a16092 report status: fi...,BWH,2008-04-14,report status: final type: surgical pathology...,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
15516,100053049,accession number: ms07r74905 report status: fi...,MGH,2007-12-12,report status: final type: surgical pathology...,1,0,0,1,0
16168,107187498,accession number: s17-35531 report status: fin...,MGH,2017-05-26,report status: final type: surgical pathology ...,1,0,0,1,0
16379,104222082,accession number: s14-312 report status: final...,MGH,2014-01-03,port status: final type: surgical pathology pa...,1,0,0,1,0
7465,105835635,accession number: s15-82321 report status: fin...,MGH,2015-12-22,report status: final type: surgical pathology ...,1,0,0,1,0


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
lymph_df_nostage.to_csv("lymph_df_nostage.csv", sep='\t')
lymph_df_withstage.to_csv("lymph_df_withstage.csv", sep='\t')

from google.colab import files
files.download('lymph_df_withstage.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
n0EMPI_df = n0_df.EMPI
n1EMPI_df = n1_df.EMPI

display(n0EMPI_df)
display(n1EMPI_df)

n0EMPI_df.to_csv("n0_EMPI_220422.csv", sep='\t')
n1EMPI_df.to_csv("n1_EMPI_220422.csv", sep='\t')

1        100006532
2        100006636
3        100001308
8        100081580
11       100086908
           ...    
16648    111017906
16649    111104018
16651    111108962
16653    111085770
16655    111061498
Name: EMPI, Length: 7531, dtype: int64

92       100138531
156      107079507
187      107948843
201      108849667
216      105059147
           ...    
16426    100895546
16503    101968130
16513    102028850
16577    113351226
16589    114288418
Name: EMPI, Length: 277, dtype: int64

In [ ]:
# LN metastasis labeling
searchforNOMETAS = ["lymph node negative for",
                    "lymph node, negative for",
                    "lymph node; negative for",
                    "lymph node: negative for",
                    "lymph nodes negative for",
                    "lymph nodes, negative for",
                    "lymph nodes: negative for",
                    "lymph nodes; negative for",

                    "lymph node; no tumor",
                    "lymph node; no malign",
                    "lymph node; no metas",
                    "lymph node; no carcin",
                    "lymph node; no evidence of",

                    "lymph nodes with no evidence of metas",

                    "benign lymph node",
                  
                    "no evidence of malignan",
                    
                    "lymph node with no tumor",
                    "lymph nodes with no tumor",
                    
                    "lymph node with no carcinoma",
                    "lymph nodes with no carcinoma",

                    "uninvolved by invasive carcinoma regional lymph nodes",

                    "no definite lymph",
                    "no definite perin"
                  ];
#metas_df = lymph_df[~lymph_df['Report_Text'].str.contains('|'.join(searchforNOMETAS))]
nometas_df = lymph_df_nostage[lymph_df_nostage['Report_Text'].str.contains('|'.join(searchforNOMETAS))]
display(nometas_df.shape)
Xnometas_df = lymph_df_nostage[~lymph_df_nostage['Report_Text'].str.contains('|'.join(searchforNOMETAS))]
display(Xnometas_df.shape)
search_report = 10
display("Xnometas example report", Xnometas_df.Report_Text.iloc[search_report])
display("Xnometas example report", Xnometas_df.Report_Text.iloc[search_report+1])
display("Xnometas example report", Xnometas_df.Report_Text.iloc[search_report+2])

searchforMETAS = ["lymph node: metas",
                  "lymph nodes: metas"];
metas_df = lymph_df_nostage[lymph_df_nostage['Report_Text'].str.contains('|'.join(searchforMETAS))]
display("metas_df:", metas_df.shape)
display(metas_df.Report_Text.iloc[0])

searchforX = ["n2 ", "n2m", "n2\.", "n2,", "n2:", "n2\(", "n2\)"];
# lymph nodes with no evidence of metas = 10
# no lymph nodes present = 21
X_df = lymph_df_nostage[lymph_df_nostage['Report_Text_noAN'].str.contains('|'.join(searchforX))]
display(X_df.shape)
display(X_df.Report_Text.iloc [5])



# Overlap between datasets
#list(set(nometas_df.EMPI) & set(xn1x_df.EMPI))

(3914, 8)

(2850, 8)

'Xnometas example report'

'accession number: bs10m02862 report status: final type: surgical pathology specimen type: prostate, radical resection procedure date: 01/21/2010 ordering provider: jim c hu m.d., m.p.h. case: bs-10-m02862 patient: richard desmond resident: douglas i. lin, m.d., ph.d. pathologist: william r welch, m.d. pathologic diagnosis: a. prostate, radical prostatectomy: prostatic adenocarcinoma (0.5 cm in greatest dimension), gleason grade 3+3 =6, present in both sides. no lymphovascular invasion is identified. tumor is not present in extraprostatic soft tissue. tumor is not present in the seminal vesicles. tumor is not present at the soft tissue resection margin. ajcc stage (7th edition): t2c nx mx. clinical data: history: not provided operation: robot assisted laparoscopic nerve sparing radical prostatectomy operative findings: not provided clinical diagnosis: prostate ca tissue submitted: a/1. prostate (tissue bank) gross description: the specimen is received fresh in the frozen room, labeled 

'Xnometas example report'

...

'metas_df:'

(6, 8)

'accession number: bs12a45661 report status: final type: surgical pathology specimen type: ureter biopsy procedure date: 10/03/2012 ordering provider: jerome p richie m.d. case: bs-12-a45661 patient: urgel dacosta resident: leigh compton, m.d., ph.d. pathologist: agoston tony agoston, m.d., ph.d. pathologic diagnosis: a. distal left ureter (including fsa): portion of ureter, negative for tumor and dysplasia. b. left pelvic lymph nodes: metastatic urothelial carcinoma involving one of five lymph nodes (1:5). c. bladder and prostate, cystoprostatectomy: urothelial carcinoma, high grade (at least 2.0cm), predominantly located in the prostate with focal extension into the bladder neck. bladder with chronic inflammation and focal ulceration. the ureteral, prostatic urethral, and peri-prostatic/vesicular soft tissue resection margins are negative for tumor. no lymphovascular invasion is identified. severe chronic inflammation and giant cell reaction are present involving predominantly the ba

(0, 8)

IndexError: ignored

In [ ]:
def stage_found(string):
  stage_df = lymph_df[lymph_df['Report_Text'].str.contains('|'.join(string))]
  display(string, stage_df.shape)

stage_found(["ct1a"])
stage_found(["ct1b"])
stage_found(["ct1c"])
stage_found(["ct2a"])
stage_found(["ct2b"])
stage_found(["ct2c"])
stage_found(["pt2"])
stage_found(["ct3a"])
stage_found(["ct3b"])
stage_found(["pt3a"])
stage_found(["pt3b"])
stage_found(["pt4"])
stage_found(["ct1c", "ct2a", "ct2b", "pt2", "pt3a", "pt3b", "pt4"])
stage_found(["report status: "])
stage_found(["accession number: "])


['ct1a']

(0, 8)

['ct1b']

(0, 8)

['ct1c']

...

(6970, 8)

['report status: ']

(14597, 8)

['accession number: ']

(14597, 8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0        report status: final type: surgical pathology ...
1        report status: final type: surgical pathology ...
2         report status: final type: surgical pathology...
3        report status: final type: surgical pathology ...
5        report status: final type: surgical pathology ...
                               ...                        
16651    report status: final type: surgical pathology ...
16652    report status: final type: surgical pathology ...
16653    report status: final type: surgical pathology ...
16654    report status: final type: surgical pathology ...
16655     report status: final type: surgical pathology...
Name: Report_Text_noAN, Length: 14597, dtype: object

'lymph node, negative for'

In [ ]:
# LN metastasis labeling
searchforNOMETAS = ["lymph node negative for",
                    "lymph node, negative for",
                    "lymph node; negative for",
                    "lymph node: negative for",
                    "lymph nodes negative for",
                    "lymph nodes, negative for",
                    "lymph nodes: negative for",
                    "lymph nodes; negative for",

                    "lymph node; no tumor",
                    "lymph node; no malign",
                    "lymph node; no metas",
                    "lymph node; no carcin",
                    "lymph node; no evidence of",

                    "lymph nodes with no evidence of metas",

                    "benign lymph node",

                    #"no lymphatic invasion",
                    #"no lymphatic or vascular invasion",
                    #"no vascular or lymphatic invasion",
                    #"no lymphovascular invasion",
                    #"lymphovascular invasion: not identified",
                    #"no lymphatic vascular invasion",
                    #"lymph-vascular invasion: not identified",
                  
                    "no evidence of malignan",
                    
                    "lymph node with no tumor",
                    "lymph nodes with no tumor",
                    
                    "lymph node with no carcinoma",
                    "lymph nodes with no carcinoma",

                    #"no evidence of perineural or lymphovascular involvement",
                    #"no lymphatic/vascular invasion",
                    #"no perineural or lymphovascular invasion",
                    #"no evidence of lymphovascular involvement",
                    #"neither perineural nor lymphovascular invasion",
                    #"lymphovascular invasion is not identified",
                    #"lymphovascular invasion and perineural invasion are not identified",
                    #"no definitive lymphovascular invasion",
                    #"no definite lymphovascular invasion",
                    #"no definite angiolymphatic invasion",
                    #"lymphovascular invasion is not seen",
                    #"no definitive lymphovascular",
                    "uninvolved by invasive carcinoma regional lymph nodes",
                    #"lymphatic invasion is not seen",
                    #"lymphovascular invasion and extraprostatic extension are not identified",
                    #"lymphovascular invasion: absent",
                    #"lymphatic/vascular invasion is not identified",
                    #"no lymphovascular or perineural invasion is identified",

                    "no definite lymph",
                    "no definite perin"
                  ];
#metas_df = lymph_df[~lymph_df['Report_Text'].str.contains('|'.join(searchforNOMETAS))]
nometas_df = lymph_df_nostage[lymph_df_nostage['Report_Text'].str.contains('|'.join(searchforNOMETAS))]
display(nometas_df.shape)
Xnometas_df = lymph_df_nostage[~lymph_df_nostage['Report_Text'].str.contains('|'.join(searchforNOMETAS))]
display(Xnometas_df.shape)
search_report = 10
display("Xnometas example report", Xnometas_df.Report_Text.iloc[search_report])
display("Xnometas example report", Xnometas_df.Report_Text.iloc[search_report+1])
display("Xnometas example report", Xnometas_df.Report_Text.iloc[search_report+2])

searchforMETAS = ["lymph-vascular invasion: present",
                  "lymphovascular invasion: present",
                  "lymph node: metas",
                  "lymph nodes: metas"];
metas_df = lymph_df_nostage[lymph_df_nostage['Report_Text'].str.contains('|'.join(searchforMETAS))]
display("metas_df:", metas_df.shape)
#display(metas_df.Report_Text.iloc[2])

searchforX = ["no definitive perineural invasion or lymphovascular invasion is identified"];
# no definitive lymphovascular invasion = 569
# no definite lymphovascular invasion = 126
# no definite angiolymphatic invasion = 6
# lymphovascular invasion is not seen = 185
# no definitive lymphovascular = 683
# lymphatic invasion is not seen = 31
# "lymphovascular invasion and extraprostatic extension are not identified" = 2
# lymphovascular invasion: absent = 40
# lymph nodes with no evidence of metas = 10
# lymphatic/vascular invasion is not identified = 3
# no definite lymph = 8
# no definite perin = 6
# no lymphovascular or perineural invasion is identified = 19

# ASK MADHUR - uninvolved by invasive carcinoma regional lymph nodes = 461

X_df = Xnometas_df[Xnometas_df['Report_Text'].str.contains('|'.join(searchforX))]
display(X_df.shape)
display(X_df.Report_Text.iloc [2])

#searchforAJCC = [" n0 ", " n1 ", " nx ", "no mx"];
#noAJCC_df = lymph_df[~lymph_df['Report_Text'].str.contains('|'.join(searchforAJCC))]
#display(noAJCC_df.shape)
#display(noAJCC_df.Report_Text.iloc [5])


# string_found("n0",)
#searchforxn0x = [" n0 "];
#xn0x_df = lymph_df[lymph_df['Report_Text'].str.contains('|'.join(searchforxn0x))]
#searchforxn1x = [" n1 "];
#xn1x_df = lymph_df[lymph_df['Report_Text'].str.contains('|'.join(searchforxn1x))]
#searchforxnxx = [" nx "];
#xnxx_df = lymph_df[lymph_df['Report_Text'].str.contains('|'.join(searchforxnxx))]

#searchforMETAS = ["lymph node; no evidence of"];
#metas_df = lymph_df[lymph_df['Report_Text'].str.contains('|'.join(searchforMETAS))]

#secondscreen = ["lymph nodes: negative for tumor",
#                "lymph nodes: negative for malign",
#                "lymph nodes: negative for carcin",
#                "lymph nodes: negative for metas"]
#display(metas_df.shape)
#display(metas_df.Report_Text.iloc[1])
#display(metas_df[metas_df['Report_Text'].str.contains('|'.join(secondscreen))].shape)
#display(metas_df[~metas_df['Report_Text'].str.contains('|'.join(secondscreen))].Report_Text.iloc[0])

# Overlap between datasets
#list(set(nometas_df.EMPI) & set(xn1x_df.EMPI))

(4913, 7)

(794, 7)

'Xnometas example report'

'accession number: ms09r34960 report status: final type: surgical pathology pathology report: ms-09-r34960 case: ms-09-r34960 patient: don hawley date taken: 6/2/2009 source care unit: mgh inpatient path subspecialty service: genito-urinary results to: douglas middleton dahl, m.d. resident: ricard masia, m.d. signed out by: peter m sadow, m.d., ph.d. clinical data: prostate cancer. final diagnosis: prostate, prostatectomy (60.3 gm): adenocarcinoma, gleason score 3 + 3 = 6/10, bilateral (pt2c), involving <5% of examined tissue, confined to the prostate (see note). note: all surgical margins, seminal vesicles and vasa deferentia are negative for carcinoma. no perineural or lymphovascular invasion is identified. gross description: received fresh, labeled don hawley, #468-32-24 and "prostate", is a 60.3 gram, 5.5 x 5.0 x 4.5 cm radical prostatectomy specimen with unremarkable external surface. the left vas deferens measures 3.4 cm in length and 0.3 cm in diameter, the right vas deferens me

'Xnometas example report'

...

'accession number: ms09r79188 report status: final type: surgical pathology pathology report: ms-09-r79188 case: ms-09-r79188 patient: jeffrey kroesen date taken: 12/8/2009 source care unit: mgh inpatient path subspecialty service: genito-urinary results to: douglas middleton dahl, m.d. resident: yeowon kim, m.d. signed out by: colleen a lamb, m.d. clinical data: prostate cancer. final diagnosis: prostate, radical prostatectomy: adenocarcinoma, gleason score 3+3 = 6/10, very focally involving the right and left lobes, confined to the prostate (pt2). note: the vasa deferentia and seminal vesicles are uninvolved by tumor. all resection margins are negative. no perineural or lymphovascular invasion is seen. gross description: received fresh, labeled jeffrey kroesen, #475-86-40 and "prostate", is a 49.1 gram, 5.2 x 4.5 x 4.5 cm prostate with attached grossly unremarkable 4.5 cm in length x 0.9 cm in diameter left vas deferens, 4.5 x 2.0 x 1.1 cm fragmented left seminal vesicle, 4.7 cm in l

'metas_df:'

(67, 7)

(2, 7)

IndexError: ignored

In [ ]:
searchfor = ['metas', 'malignan','tumor','invasion','confined']
final_df = lymph_df[lymph_df['Report_Text'].str.contains('|'.join(searchfor))]

In [ ]:
# check for n+digit 

In [ ]:
final_df[final_df['Report_Text'].str.contains('meta')].Report_Text.iloc[4]

'accession number: bs19f04711 report status: updated type: surgical pathology procedure date: 01/24/2019 ordering provider: adam stuart kibel m.d. case: bs-19-f04711 patient: charles risio brigham and women\'s hospital department of pathology 75 francis street, boston, ma 02115 clia license no.: 22d0705149 laboratory director: dr. christopher d.m. fletcher physician: adam stuart kibel, m.d. procedure date: 01/24/2019 ========== revised report ========== resident: andres acosta, m.d. pathologist: michelle s. hirsch, m.d., ph.d. pathologic diagnosis: a. prostate, seminal vesicles and periprostatic fat, prostatectomy: prostatic adenocarcinoma. tumor fecally invades the extraprostatic soft tissue. resection margins are negative for tumor. see synoptic report below for detailed findings. b. additional left lateral margin: fibroconnective tissue, negative for tumor. c. right pelvic lymph nodes: seven (7) lymph nodes, negative for tumor. d. left pelvic lymph nodes: metastatic prostatic adenoc

In [ ]:
final_df.shape

(14557, 4)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from time import time
from sklearn.cluster import KMeans


In [ ]:
from nltk.stem.snowball import EnglishStemmer

stemmer = EnglishStemmer()
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [ ]:
def clean_data(df):
  data = df.copy()
  for i,row in data.iterrows():
    sen = row['Report_Text']
    sen = re.sub("[^a-zA-Z0-9]+", ".",sen).lower()
    sen = sen.split('.')
    sen = " ".join(sen)
    sen =  sen 
    data.loc[i,'text'] = sen
  return data['text']

In [ ]:
data = clean_data(final_df)

In [ ]:
vectorizer = TfidfVectorizer(
    analyzer=stemmed_words,
    max_df=0.7,
    min_df=2,
    ngram_range = (2,2),
    use_idf=True,
)
X = vectorizer.fit_transform(data)

In [ ]:
X.shape

(14557, 9442)

In [ ]:
print("Performing dimensionality reduction using LSA")
t0 = time()
# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD(1000)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X_low = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print(
    "Explained variance of the SVD step: {}%".format(int(explained_variance * 100))
)

print()

Performing dimensionality reduction using LSA
done in 41.869687s
Explained variance of the SVD step: 88%



In [ ]:
X_low.shape

(14557, 1000)

In [ ]:
km2 = KMeans(
    n_clusters=2,
    init="k-means++",
    max_iter=100,
    n_init=1
)

In [ ]:
print("Clustering sparse data with %s" % km2)
t0 = time()
km2.fit(X)
print("done in %0.3fs" % (time() - t0))

Clustering sparse data with KMeans(max_iter=100, n_clusters=2, n_init=1)
done in 0.153s


In [ ]:
print("Top terms per cluster:")
order_centroids = km2.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(2):
    print("Cluster %d:" % i, end="")
    for ind in order_centroids[i, :20]:
        print(" %s" % terms[ind], end="")
    print()

Top terms per cluster:
Cluster 0: frag micro ess slice rss pelvic fragment consist for apex one biopsi lp rp base candid each soft multi ph
Cluster 1: quadrant obtur pelvic inferior cassett superior frozen dissect measur md malign evid distal there serial bladder proxim unremark follow excis


In [ ]:
from sklearn.metrics import silhouette_samples

In [ ]:
sample_silhouette_values = silhouette_samples(X, km2.labels_)
print(f"Silhouette values:")
silhouette_values = []
for i in range(2):
    cluster_silhouette_values = sample_silhouette_values[km2.labels_ == i]
    silhouette_values.append(
        (
            i,
            cluster_silhouette_values.shape[0],
            cluster_silhouette_values.mean(),
            cluster_silhouette_values.min(),
            cluster_silhouette_values.max(),
        )
    )
silhouette_values = sorted(
    silhouette_values, key=lambda tup: tup[2], reverse=True
)
for s in silhouette_values:
    print(
        f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
    )

Silhouette values:
    Cluster 0: Size:5367 | Avg:0.28 | Min:0.09 | Max: 0.41
    Cluster 1: Size:9190 | Avg:0.06 | Min:-0.10 | Max: 0.11


In [ ]:
km4 = KMeans(
    n_clusters=2,
    init="k-means++",
    max_iter=100,
    n_init=1
)

In [ ]:
print("Clustering sparse data with %s" % km4)
t0 = time()
km4.fit(X_low)
print("done in %0.3fs" % (time() - t0))
print()

Clustering sparse data with KMeans(max_iter=100, n_clusters=2, n_init=1)
done in 0.468s



In [ ]:
original_space_centroids = svd.inverse_transform(km4.cluster_centers_)
print("Top terms per cluster:")
order_centroids = original_space_centroids.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(2):
    print("Cluster %d:" % i, end="")
    for ind in order_centroids[i, :20]:
        print(" %s" % terms[ind], end="")
    print()

Top terms per cluster:
Cluster 0: quadrant obtur pelvic inferior superior cassett frozen dissect measur md malign evid distal there serial bladder proxim follow unremark excis
Cluster 1: frag micro ess slice rss pelvic fragment consist for apex one biopsi lp rp base candid each soft multi ph


In [ ]:
sample_silhouette_values = silhouette_samples(X_low, km4.labels_)
print(f"Silhouette values:")
silhouette_values = []
for i in range(2):
    cluster_silhouette_values = sample_silhouette_values[km4.labels_ == i]
    silhouette_values.append(
        (
            i,
            cluster_silhouette_values.shape[0],
            cluster_silhouette_values.mean(),
            cluster_silhouette_values.min(),
            cluster_silhouette_values.max(),
        )
    )
silhouette_values = sorted(
    silhouette_values, key=lambda tup: tup[2], reverse=True
)
for s in silhouette_values:
    print(
        f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
    )

Silhouette values:
    Cluster 1: Size:5380 | Avg:0.30 | Min:0.10 | Max: 0.44
    Cluster 0: Size:9177 | Avg:0.07 | Min:-0.11 | Max: 0.12


In [ ]:
labels = km4.predict(X_low)

In [ ]:
final_df['y_raw_kmeans'] = labels

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_df[final_df['y_raw_kmeans'] == 1].sample(1).Report_Text.values[0]

'accession number: bs10w33354 report status: final type: surgical pathology specimen type: soft tissue tumor, extensive resection procedure date: 07/30/2010 ordering provider: graeme steele m.d. case: bs-10-w33354 patient: walter becht resident: brooke e. lane, m.d. pathologist: jeffrey f krane, m.d., ph.d.; michelle s hirsch, m.d., ph.d. pathologic diagnosis: a. left ureter, margin (including fsa): segment of ureter with chronic inflammation, negative for tumor (best seen on original frozen section slide). b. right ureter, margin (including fsb): segment of ureter, negative for tumor. c. cystoprostatectomy: bladder: focal, residual papillary urothelial carcinoma, high grade (former who grade 3 of 3) (best seen in slides c2, c4, and c8), with focal extension into cystitis glandularis. no invasive tumor identified. the urethral and radial soft tissue resection margins are negative for tumor. no lymphovascular invasion. prostate: prostatic adenocarcinoma, microscopic foci up to 0.5 cm in

In [ ]:
final_df[final_df['y_raw_kmeans'] == 0].sample(1).Report_Text.values[0]

'accession number: ms10x46648 report status: final type: surgical pathology pathology report: ms-10-x46648 case: ms-10-x46648 patient: george martwichuck date taken: 7/21/2010 source care unit: mgh inpatient path subspecialty service: genito-urinary results to: francis james mcgovern, m.d. resident: darcy kerr, m.d. signed out by: colleen a lamb, m.d. clinical data: history of prostate cancer. final diagnosis: prostate, radical prostatectomy: adenocarcinoma, gleason score 3 + 4 = 7/10, focally involving all four quadrants, confined to the prostate (pt2); see note. note: all resection margins are free of tumor. lymphovascular invasion is not seen. gross description: received fresh, labeled george martwichuck, #491-67-67, and "prostate", is a 57.5 g, 4.7 x 4.5 x 4.3 cm radical prostatectomy specimen with an unremarkable external surface. the left seminal vesicle measures 1.5 x 0.9 x 0.5 cm, the right seminal vesicle measures 1.4 x 1.0 x 0.4 cm, and they are both unremarkable. the left va

In [ ]:
  # if opts.n_components:
  #     original_space_centroids = svd.inverse_transform(km.cluster_centers_)
  #     order_centroids = original_space_centroids.argsort()[:, ::-1]
  # else:

In [ ]:
# Try word2vec

**K-means using word embedding**

In [ ]:
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization
from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [ ]:
data = final_df.copy()


new_df = clean_data(data)
new_df.head()

0    accession number s02k17074 report status final...
1    accession number s9905462c report status final...
2    accession number bs05e05528 report status fina...
3    accession number s9935420s report status final...
5    accession number s99t17262 report status final...
Name: text, dtype: object

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(new_df)]
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

In [ ]:
emb_X = []
for i in range(len(new_df)):
  text = new_df.iloc[i]
  emb_X.append(model.infer_vector(text))

In [ ]:
emb_X = np.array(emb_X)

In [ ]:
def mbkmeans_clusters(
	X, 
    k, 
    mb, 
    print_silhouette_values, 
):
    """Generate clusters and print Silhouette metrics using MBKmeans

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(
	X=emb_X,
    k=2,
    mb=10,
    print_silhouette_values=True,
)
# df_clusters = pd.DataFrame({
#     "text": docs,
#     "tokens": [" ".join(text) for text in tokenized_docs],
#     "cluster": cluster_labels
# })

For n_clusters = 2
Silhouette coefficient: 0.13
Inertia:23188.796875
Silhouette values:
    Cluster 0: Size:5071 | Avg:0.14 | Min:-0.02 | Max: 0.28
    Cluster 1: Size:9486 | Avg:0.13 | Min:-0.05 | Max: 0.26


In [ ]:
model.docvecs.most_similar([original_space_centroids[0]])

ValueError: ignored

In [ ]:
model.docvecs.most_similar([original_space_centroids[1]])

In [ ]:
  new_df.iloc[7568]

In [ ]:
new_df.iloc[7885]

In [ ]:
new_df.iloc[11698]

In [ ]:
new_df.iloc[10785]